## Домашка 2. Профилирование пользователей



In [299]:
# загружаем необходимые библиотеки

import pandas as pd

from gensim.corpora.dictionary import Dictionary

import re
import numpy as np
from nltk.corpus import stopwords

from razdel import tokenize

import pymorphy2

from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import itertools

import matplotlib.pyplot as plt

%matplotlib inline

In [116]:
# смотрим данные по статьям

news = pd.read_csv("articles.csv")
print(news.shape)
news.head(13)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...
3,4898,Главный тренер «Кубани» Юрий Красножан прокомм...
4,4899,Решением попечительского совета владивостокско...
5,4900,И.о. главного тренера Вячеслав Буцаев прокомме...
6,4901,Запорожский «Металлург» дома потерпел разгромн...
7,4902,Сборная США одержала победу над Австрией со сч...
8,4903,Бывший защитник сборной России Дарюс Каспарайт...
9,4904,Полузащитник ЦСКА Зоран Тошич после победы над...


In [117]:
# данные какой пользователь какими статьями интересовался

users = pd.read_csv("users_articles.csv")
users.head(13)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"
3,u101138,"[5933, 6186, 5055, 6977, 5206, 488389]"
4,u108248,"[707, 1144, 2532, 2928, 3133, 324592]"
5,u106662,"[323868, 323426, 324267, 322426, 324104, 1550]"
6,u105949,"[293138, 294471, 295012, 294736, 293949, 3544]"
7,u102457,"[6928, 5009, 6940, 7629, 7644, 512736]"
8,u104124,"[322838, 324699, 322991, 322120, 324327, 472331]"
9,u101386,"[7827, 6427, 7394, 7151, 6335, 487254]"


In [121]:
# загружаем слова не несущие смысла - предлоги всякие и т.д.

stopword_ru = stopwords.words('russian')
len(stopword_ru)

151

In [122]:
morph = pymorphy2.MorphAnalyzer()

In [123]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [124]:
%%time
# чистим текст 
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

Wall time: 26.3 s


In [125]:
news['title']

0        заместитель председателяnправительства рфnсерг...
1        матч  финала кубка россии по футболу был приос...
2        форвард авангарда томаш заборский прокомментир...
3        главный тренер кубани юрий красножан прокоммен...
4        решением попечительского совета владивостокско...
                               ...                        
26995    ученые токийского университета морских наук и ...
26996    главой кафедры отечественной истории xx века и...
26997    американские ученые уточнили возраст расположе...
26998    за последние  лет тропический углеродный цикл ...
26999    у живших примерно  тыс лет назад на территории...
Name: title, Length: 27000, dtype: object

In [126]:
%%time
# леммитизация текста - перевод слов в именительный падеж, единственное число или типа того...
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

Wall time: 2min 36s


In [128]:
news['title']

0        [заместитель, председатель, правительство, рф,...
1        [матч, финал, кубок, россия, футбол, приостано...
2        [форвард, авангард, томаш, заборский, прокомме...
3        [главный, тренер, кубань, юрий, красножанин, п...
4        [решение, попечительский, совет, владивостокск...
                               ...                        
26995    [учёный, токийский, университет, морской, наук...
26996    [глава, кафедра, отечественный, история, xx, в...
26997    [американский, учёный, уточнить, возраст, расп...
26998    [последний, год, тропический, углеродный, цикл...
26999    [жить, примерно, тыс, год, назад, территория, ...
Name: title, Length: 27000, dtype: object

In [129]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]

In [131]:
# texts

In [132]:
# Create a corpus from a list of texts

common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

In [135]:
common_dictionary[100]

'выбросить'

In [136]:
len(common_dictionary)

141492

In [138]:
# common_corpus

In [139]:
%%time
from gensim.models import LdaModel
# Train the model on the corpus.
lda = LdaModel(common_corpus, 
               num_topics=25, # количество кластеров, т.е. тем, на которые мы хотим разбить наши тексты 
               id2word=common_dictionary)#, passes=10)

Wall time: 33.4 s


In [140]:
# lda

In [141]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'свой', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'nnnn', 'провести', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'работа', 'сказать', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'мочь', 'играть', 'ещё', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'это', 'хороший']


[(3, 0.19652262),
 (6, 0.036923707),
 (8, 0.49084795),
 (20, 0.15705821),
 (21, 0.100806095)]

In [142]:
# other_texts

In [143]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: рейс аэропорт соцсеть телефонный индонезия екатеринбург хабаровский
topic_1: ракета активность доклад офицер характерный израиль который
topic_2: год это который компания рубль млрд млн
topic_3: фонд дональд университет рейтинг фильм место определение
topic_4: это год который nn правительство президент россия
topic_5: британский великобритания бензин австралия австралийский арабский макаров
topic_6: восток больной зарегистрировать орден подниматься значительно проявляться
topic_7: год который земля это доля поверхность новый
topic_8: это год который всё весь свой очень
topic_9: общество расчёт сигнал рекомендовать ресурс избегать законопроект
topic_10: лётчик грунт акционер кольцо киргизия пролететь миля
topic_11: снижение дыра этаж одежда мэй снизить значительно
topic_12: таиланд су гонконг жюри эстонский гвинея предназначение
topic_13: это год который россия мочь свой страна
topic_14: статья конкурс тыс nn год сообщаться предприниматель
topic_15: тело суд год район сотрудник

In [144]:
# функиця представляет векторное представление новости

#text = news['title'].iloc[0]

def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [145]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.0,0.0,0.0,0.077171,0.104139,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.051096,0.0,0.000000,0.000000,0.729822,0.0,0.0
1,4896,0.0,0.0,0.0,0.252689,0.000000,0.0,0.000000,0.000000,0.000000,...,0.037859,0.687437,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0
2,4897,0.0,0.0,0.0,0.196540,0.000000,0.0,0.036924,0.000000,0.490822,...,0.000000,0.000000,0.0,0.000000,0.0,0.157066,0.100807,0.000000,0.0,0.0
3,4898,0.0,0.0,0.0,0.040803,0.123514,0.0,0.000000,0.349718,0.365235,...,0.000000,0.046023,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0
4,4899,0.0,0.0,0.0,0.055078,0.631808,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.167925,0.0,0.0


In [146]:
# переходим к работе с пользователями

users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [147]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

In [149]:
# на примере одного документа смотрим какие темы в нем затронуты

doc_dict[293622]

array([0.        , 0.        , 0.        , 0.        , 0.30825761,
       0.        , 0.02090073, 0.        , 0.26695648, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.05267735, 0.        , 0.        , 0.        ,
       0.12734206, 0.        , 0.        , 0.21233605, 0.        ])

In [150]:
# doc_dict

In [151]:
user_articles_list = users['articles'].iloc[33]

In [253]:
# на примере пользователя выбираем статьи, которыми он интересовался

user_articles_list

'[323329, 321961, 324743, 323186, 324632, 474690]'

In [310]:
# Датасет готов - можно попробовать обучить модель. Загрузим нашу разметку

target = pd.read_csv("users_churn.csv")
# target.head(3)

## Get_User_Embedding

In [340]:
def get_user_embedding_mean(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.mean(user_vector, 0)
    return user_vector

def get_user_embedding_median(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.median(user_vector, 0)
    return user_vector

def get_user_embedding_max(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.max(user_vector, 0)
    return user_vector

In [341]:
def metrix_(user_embeddings):
    user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
    user_embeddings['uid'] = users['uid'].values
    user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
    # user_embeddings.head(3)

    X = pd.merge(user_embeddings, target, 'left')

    #разделим данные на train/test
    X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                        X['churn'], random_state=0)

    logreg = LogisticRegression()
    #обучим 
    logreg.fit(X_train, y_train)

    #наши прогнозы для тестовой выборки
    preds = logreg.predict_proba(X_test)[:, 1]

    # считаем Precision, Recall, F_score

    precision, recall, thresholds = precision_recall_curve(y_test, preds)
    fscore = (2 * precision * recall) / (precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f, roc_auc_score=%.3f' % (thresholds[ix], 
                                                                            fscore[ix],
                                                                            precision[ix],
                                                                            recall[ix],
                                                                            roc_auc_score(y_test, preds)))


In [343]:
# Теперь получим эмбединги для всех пользователей и проверим их качество на конкретной downstream-задаче

user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding_mean(x), 1)])
print('mean:')
metrix_(user_embeddings)

user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding_median(x), 1)])
print('median:')
metrix_(user_embeddings)

user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding_max(x), 1)])
print('max:')
metrix_(user_embeddings)

mean:
Best Threshold=0.281794, F-Score=0.711, Precision=0.662, Recall=0.767, roc_auc_score=0.957
median:
Best Threshold=0.257147, F-Score=0.780, Precision=0.713, Recall=0.861, roc_auc_score=0.967
max:
Best Threshold=0.371561, F-Score=0.803, Precision=0.798, Recall=0.808, roc_auc_score=0.971


## Вывод:

Лучшие метрики получаются, если считаетмы МАКСИМУМ

Думаю, это потому-что максимум - это тема, которая больше всего интересовала данного пользователся. 
Т.е. мы опираем на лучшие данные, соответственно месрики получаются лучше.